In [72]:
from sympy import *
from sympy import I as j #import complex number as j
from sympy.solvers.solveset import linsolve, nonlinsolve
from sympy.parsing.latex import parse_latex
from sympy.parsing.sympy_parser import parse_expr
from IPython.display import display, Math
import ipywidgets as widgets
#define symbols
#dont use i as a symbol, because it is reserved for the imaginary unit!!!
R=symbols("R_{0:7}", real = True)
I=symbols("I_{0:7}")
i=symbols("i_{0:7}")
C=symbols("C_{0:7}", real=True)
L=symbols("L_{0:7}")
V=symbols("V_{0:7}")
Vin = symbols("V_in")
Vr = symbols("V_i*n")
t =symbols("t", real=True)
s = symbols("s")
omega=symbols("omega")
#j=symbols("j")
Pr3 = symbols("P_R3")

In [2]:
# out = widgets.Output(layout={'border': '1px solid black'})
outIPy = widgets.Output(layout={'border': '2px solid black'})
initNumOfBoxes = {"new" : 2}
numOfR = 5
textBox = []
subsEqNum = 1

pEq = []
linEq = []
lapEq = []


style = {'description_width': 'initial'}
layout = widgets.Layout(width='auto', height='40px')

def drawTBox(change): #dinamically draw pEq input text boxes
    out.clear_output()
    
    global numOfBoxes   #global needed because without it the next statement wouldn't update the global variable
    numOfBoxes = int(change["new"])
    
    global textBox
    textBox = []
    for i in range(numOfBoxes):
        textString = "Eq" + str(i+1) + ":"
        textBox.append(widgets.Text(description=textString))
        #print(numOfBoxes)
        with out:
            display(textBox[i])

drawTBox(initNumOfBoxes)

dropdown = widgets.Dropdown(
    options=['1', '2', '3', '4'],
    value = str(initNumOfBoxes["new"]),
    description='Number of pEqs:',
    disabled=False,
    style=style
)

#def tes(change):
#    print(change["new"])

def changeSubs(change):  #change for which pEqs to substitute values
    global subsEqNum
    subsEqNum = change["new"]

#drawTBox(int(dropdown.value))

radioButton = widgets.RadioButtons(
    options=[('parsed equation',1), ('linsolve equation',2), ('custom equation',3)],
    disabled=False,
    layout=layout
)

radioLabel = widgets.Label('Substitute or inv. laplace transform:', layout=widgets.Layout(width='auto'))

button1 = widgets.Button(
    description='Parse equations',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

button2 = widgets.Button(
    description='LinSolve for I',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

button3 = widgets.Button(
    description='Substitute in equations',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button4 = widgets.Button(
    description='Clear Subs Val',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button5 = widgets.Button(
    description='Clear Output',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button6 = widgets.Button(
    description='Inverse Laplace Trans',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

button7 = widgets.Button(
    description='Calculate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='hello',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
   
)

#pEq = [Eq((R[1]+1/(C[1]*s))*I[1]-(1/(C[1]*s))*I[2],V[1]),
#           Eq(-1/(C[1]*s)*I[1]+(R[2]+1/(C[1]*s))*I[2],0)]
#display(pEq[0])
#for i in pEq:
 #   display(i)
#(R[1]+1/(C[1]*s))*I[1]-(1/(C[1]*s))*I[2]

#def handle_submit(sender):
 #   print(textBox.value)
  #  print(sender)

def handle_button7(a):  #custom equation calc
    global cEq
    cEq = []
    boxInLhs = customEqBox.value 
    boxInRhs = boxInLhs
    if customEqBox.value:
        
        for key, val in {"pEq":pEq, "linEq":linEq, "lapeq":lapEq}.items():
            
            for num in range(len(val)):
            
                pEqLhs = str(val[num].lhs).replace("{", "").replace("}", " ") #because parse_expre dosen't like I_{x}
                pEqRhs = str(val[num].rhs).replace("{", "").replace("}", " ")
                boxInLhs = boxInLhs.replace(key + "[" + str(num) + "]", pEqLhs)
                boxInRhs = boxInRhs.replace(key + "[" + str(num) + "]", pEqRhs)
            
        cEq.append(Eq(parse_expr(boxInLhs), parse_expr(boxInRhs)))
    
    #elif boxIn.find("linEq[")
    #for n in [pEq] 
    
    for eq in cEq:
        outIPy.append_display_data(eq)

def handle_button6(a):  #laplace transform the solved linEqs 
    global lapEq
    lapEq = []
    if subsEqNum == 1: #for parsed pEqs
        eqNum = 0

        #print(subsEqNum)
    elif subsEqNum == 2: #for linsolve linEqs
        eqNum = 1

    elif subsEqNum == 3: #for custom equations
        eqNum = 2
        
    for n, res in enumerate([pEq, linEq, cEq][eqNum]):
        lapEq.append(Eq(i[n+1], expand(inverse_laplace_transform(res.rhs,s,t)))) #inv laplace dosent work for equalities
        outIPy.append_display_data(lapEq[n])


def handle_button5(a):  #clear output
    outIPy.clear_output()
    
#def handle_button4(a): #clear substitute input field values

def handle_button3(a):  #substitute values parser
    global pEq
    if subsEqNum == 3:
        forRange = range(len(cEq))
    else:
        forRange = range(len(pEq))
        
    for nEq in forRange:
        for key, textField in valueItems.items():
            for num, textF in enumerate(textField):
                if textF.value != "":
                    #print(key)
                    #print(textF.value)
                    if key=="L":
                        mult = 10**-3
                    elif key=="C":
                        mult = 10**-6 
                    else:
                        mult = 1
                    #print(key + "_{" + str(n) + "}")
                    #print(int(textF.value))
                    
                    bnum = "{" + str(num) + "}" # because Vin and custom eqs dosent need the braces {}
                    if key=="Vin":
                        bnum = "in"
                        key = "V"
                    
                    if subsEqNum == 1: #for parsed pEqs
                        pEq[nEq] = pEq[nEq].subs(key + "_" + str(bnum), parse_expr(textF.value)*mult)
                        
                        #print(subsEqNum)
                    elif subsEqNum == 2: #for linsolve linEqs
                        linEq[nEq] = linEq[nEq].subs(key + "_" + str(bnum), parse_expr(textF.value)*mult)
                        
                    elif subsEqNum == 3: #for custom equations
                        cEq[nEq] = cEq[nEq].subs(key + "_" + str(num), parse_expr(textF.value)*mult)
                        display()
                        #print(subsEqNum)
            
        if subsEqNum == 1: #for parsed pEqs
            outIPy.append_display_data(pEq[nEq])
        elif subsEqNum == 2: #for linsolve linEqs
            outIPy.append_display_data(linEq[nEq])
        elif subsEqNum == 3: #for custom equations
            outIPy.append_display_data(cEq[nEq])
                    #print(eq)
    #display(pEq[0])


def handle_button2(a):  #linear solver
    iList = []
    global linEq
    for n, eq in enumerate(pEq):
        iList.append(I[n+1])
        
    linEq = linsolve(pEq,iList).args[0]
    linEq = list(linEq)  #needed as list in other functions
    #outIPy.append_display_data(type(linEq))
    outIPy.append_display_data("result:")
    for n, res in enumerate(linEq):
        linEq[n] = Eq(I[n+1], linEq[n])
        outIPy.append_display_data(Math(latex(linEq[n]))) #pretty print
    
    
def handle_button1(a):  #input parser
    global pEq
    latex_string = []
    pEq = []

    for i in range(numOfBoxes):
        lstring = textBox[i].value.replace(r"\ ", "") # remove blank spaces
        lstring = parse_latex(lstring)
        #lstring = lstring.subs(j, i)
        lstring = lstring.subs("V_{i*n}", Vin) # multiple charachter subscripts are parsed incorectly
        pEq.append(lstring)
        outIPy.append_display_data(pEq[i])
        #display(type(pEq[i]))
    pEq = list(pEq)
        
valueItems = {"R": [], "C": [], "L": [], "V": [],"Vin": [], "I": []}
valueItemPlaceholder = {"R": "Ohms", "C": "uF", "L": "mH", "V": "Volts","Vin": "input eq", "I": "Amps"}
valueChildList = []
for key in valueItems:
    if key=="Vin":
        oldNumOfR = numOfR
        numOfR = 1
    for n in range(numOfR):
        if key=="Vin":
            n = "{in}"
            valueItems[key].append(widgets.Text(description="$" + "V" + "_" + str(n) + "$", placeholder=valueItemPlaceholder[key]))
        else:
            valueItems[key].append(widgets.Text(description="$" + key + "_" + str(n) + "$", placeholder=valueItemPlaceholder[key]))
    valueChildList.append(widgets.VBox(valueItems[key]))
    if key=="Vin":
        numOfR = oldNumOfR
    
box_layout = widgets.Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='500px',
                    height='',
                    flex_flow='column',
                    display='flex')
#carousel = widgets.Box(children=valueItems, layout=box_layout)
#carousel = widgets.Box(children=valueItems)

display(dropdown)
dropdown.observe(drawTBox, names="value")

display(widgets.Box(valueChildList))

display(out)
display(widgets.HBox([button1,button2,button6,button3,button4,button5]))

display("Write custom expression:  pEq[x] parsed   linEq[x] system of equation results   lapEq[x] inv. laplace results")

customEqBox = widgets.Text(placeholder="Equation")
display(widgets.HBox([customEqBox, button7]))


display(widgets.HBox([radioLabel, radioButton], layout=widgets.Layout(padding = "0px 0px 20px 0px")))
radioButton.observe(changeSubs, names="value")

display(outIPy)

button1.on_click(handle_button1)
button2.on_click(handle_button2)
button3.on_click(handle_button3)
#button4.on_click(handle_button4)
button5.on_click(handle_button5)
button6.on_click(handle_button6)
button7.on_click(handle_button7)

NameError: name 'widgets' is not defined